In [38]:
import pandas as pd
import  psycopg2
from faker import Faker
import random
import datetime


In [39]:
def conectar_banco():
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5433",
            database="prd_case",
            user="postgres",
            password="@Chupa123"
        )
        return conn
    except (Exception, psycopg2.Error) as error:
        print("Erro ao conectar ao PostgreSQL:", error)
        return None


In [40]:

# Quantidade de valores
quantidade = 1000

fake = Faker(['pt_BR'])

# Intervalos para gerar a data de nascimento
data_inicio = datetime.date(1925, 1, 1)
data_fim = datetime.date(2005, 12, 31)

sexos = []
datas_nascimento = []
datas_cadastro = []
cidades = []
ufs = []

# Listas para resultados
data_acessos = []
buyins = []
winnings = []
rakes = []

for gerar in range(quantidade):
    sexo = fake.random_element(['m', 'f'])
    data_nascimento = fake.date_between(start_date=data_inicio, end_date=data_fim)
    data_cadastro = fake.date_between(start_date='-3y', end_date='today')
    cidade = fake.city()
    uf = fake.state_abbr()

    # Gerando dados de Resultados
    data_acesso = fake.date_between(start_date='-1y', end_date='today')
    buyin = fake.pyfloat(left_digits=3, right_digits=2, positive=True)
    rake = fake.pyfloat(left_digits=3, right_digits=2, positive=True, max_value=buyin)
    winning = buyin - rake

    # Append para listas de Clientes
    sexos.append(sexo)
    datas_nascimento.append(data_nascimento)
    datas_cadastro.append(data_cadastro)
    cidades.append(cidade)
    ufs.append(uf)

    # Append para listas de Resultados
    data_acessos.append(data_acesso)
    buyins.append(buyin)
    winnings.append(winning)
    rakes.append(rake)

clientes = {
    'sexo': sexos,
    'data_nascimento': datas_nascimento,
    'data_cadastro': datas_cadastro,
    'cidade': cidades,
    'uf': ufs
}

resultados = {
    'data_acesso': data_acessos,
    'buyin': buyins,
    'winning': winnings,
    'rake': rakes
}

clientes = pd.DataFrame(clientes)
resultados= pd.DataFrame(resultados)



In [41]:
conexao = conectar_banco()

if conexao:
    insert_clientes = """INSERT INTO clientes (
        Sexo,
        Data_Nascimento,
        Data_Cadastro,
        Cidade,
        UF
    ) VALUES (%s, %s, %s, %s, %s)"""

    for i, row in clientes.iterrows():
        cursor = conexao.cursor()
        cursor.execute(insert_clientes, tuple(row))
        conexao.commit()
        cursor.close()

if conexao: 
    insert_resultados = """INSERT INTO resultado (
        data_acesso,
        buyin,
        winning,
        rake
    ) VALUES (%s, %s, %s, %s)"""

for i, row in resultados.iterrows():
        cursor = conexao.cursor()
        cursor.execute(insert_resultados, tuple(row))
        conexao.commit()
        cursor.close()
        
conexao.close()
